## Del 2: Varmeledning

Varmeledning er prosessen der varme overføres gjennom et materiale. Dette skjer når det er temperaturforskjeller i ulike deler av materialet. For å beregne varmeledning, så kan man løse varmelikningen.

Vi skal etter hvert løse varmelikningen, hvor vi ser på en modell for steking i en ovn. 

Før vi kommer så langt så varmer vi opp med noen øvingsoppgaver om løsning av (relevante) partielle differensialligninger. Vi henviser til forelesningene og jupyter-notatene for informasjon om hvordan likningene løses.

In [ ]:
#kodeblokk for oppgave 2